In [1]:
import sys
sys.path.append("v2_Assignment_Codes")  # Add the folder to the search path

#load data
from v2_data import get_fixed_data
from PriceProcess import price_model
from WindProcess import wind_model
from utils import generate_time_series,generate_experiment_series

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from pyomo.environ import *
from mdp import check_feasibility,sim_MDP_exp, sim_MDP

%load_ext autoreload
%autoreload 2
plt.rcParams.update({'font.size': 13})

In [2]:
data = get_fixed_data()
T = data['num_timeslots']

In [3]:
# ensure we always use the same time series for all tasks and experiments
prices,winds = generate_experiment_series()

# Task 2

In [4]:
import random

np.random.seed(42)
random.seed(42)

## Generate scenario tree

In [5]:
from sklearn.cluster import KMeans

class realization():
    def __init__(self,t,local_scenario,wind,price,prev):
        self.t = t
        self.wind = wind
        self.price = price
        self.prev = prev
        self.local_scenario = local_scenario


def generate_scenarios(wind, price, wind_previous, price_previous, stages, k=4, n_samples=100):
    scenarios = []
    scenarios.append([realization(0, 0, wind, price, None)])
    scenario_probs = [[1.0]]

    for t in range(stages):
        new_scenarios = []
        new_probs = []
        for i, realization_i in enumerate(scenarios[t]):
            wind_samples = []
            price_samples = []
            for _ in range(n_samples):
                if t == 0:
                    wind_future = wind_model(realization_i.wind, wind_previous, data)
                    price_future = price_model(realization_i.price, price_previous, wind_future, data)
                else:
                    wind_future = wind_model(realization_i.wind, realization_i.prev.wind, data)
                    price_future = price_model(realization_i.price, realization_i.prev.price, wind_future, data)
                wind_samples.append(wind_future)
                price_samples.append(price_future)

            if k < n_samples:
                samples = np.column_stack((wind_samples, price_samples))
                kmeans = KMeans(n_clusters=k, random_state=0).fit(samples)
                centroids = kmeans.cluster_centers_
                labels = kmeans.labels_

                for j in range(k):
                    centroid_wind, centroid_price = centroids[j]
                    prob = np.mean(labels == j)
                    new_scenarios.append(realization(t+1, j, centroid_wind, centroid_price, realization_i))
                    new_probs.append(scenario_probs[t][i] * prob)
            else:
                for j in range(n_samples):
                    new_scenarios.append(realization(t+1, j, wind_samples[j], price_samples[j], realization_i))
                    new_probs.append(scenario_probs[t][i] / n_samples)

        scenarios.append(new_scenarios)
        scenario_probs.append(new_probs)

    return scenarios, scenario_probs


## Code to convert sceneratios to matrix to use in MILP program

In [6]:
def scenarios_to_matrices(scenarios):
    T = len(scenarios)
    S = len(scenarios[-1])
    
    wind_matrix = np.zeros((T, S))
    price_matrix = np.zeros((T, S))
    a_set = [[]]*(T)
    a_set[0] = a_set[0] + [[i for i in range(S)]]
    for t in range(1,T):
        for s in range(len(scenarios[t])):
            span = int(len(scenarios[-1]) / len(scenarios[t]))
            prev_scenario = scenarios[t][s].prev
            a_set_indices = [_ for _ in range(s*span,(s+1)*span)]
            wind_matrix[t-1,span*s:(s+1)*span] = prev_scenario.wind
            price_matrix[t-1,span*s:(s+1)*span] = prev_scenario.price
            a_set[t] = a_set[t] + [a_set_indices]

    wind_matrix[-1] = [r.wind for r in scenarios[-1]]
    price_matrix[-1] = [r.price for r in scenarios[-1]]

    return wind_matrix, price_matrix,a_set


## Linear Program with non-anticipativity constraints

In [7]:
def solve_multi_stage_model(t_start,scenarios,a_set,scenario_probs,price,wind,h_0,e_on_0):
    T = len(scenarios)
    S = len(scenarios[-1])
    
    # Create a model
    model = ConcreteModel()
    # Declare indexed variable for the price
    model.p_grid = Var(range(T),range(S), within=NonNegativeReals,name='p_grid')
    model.e_h2p = Var(range(T),range(S), within=NonNegativeReals,bounds=(0,data['h2p_max_rate']),name='e_h2p')
    model.e_p2h = Var(range(T),range(S), within=NonNegativeReals,bounds=(0,data['p2h_max_rate']),name='e_p2h')
    model.e_on = Var(range(-1,T-1),range(S), within=Binary,name='e_on',initialize=0)

    model.h = Var(range(T),range(S), within=NonNegativeReals,bounds=(0,data['hydrogen_capacity']),name='h')

    vars = [model.p_grid,model.e_h2p,model.e_p2h,model.e_on,model.h]

    model.non_anticipativity = ConstraintList()
    for var in vars:
        for t in range(T-1):
            for a_subset in a_set[t]:
                sprime = a_subset[0]
                for s in a_subset:
                    model.non_anticipativity.add(var[t, s] == var[t, sprime])

    # Objective function
    def objective_rule(model):
        return sum(scenario_probs[s]*(price[t,s] * model.p_grid[t,s] + data['electrolyzer_cost']*model.e_on[t-1,s]) for s in range(S) for t in range(T))

    model.profit = Objective(rule=objective_rule, sense=minimize)

    model.DemandConstraint = Constraint(range(T),range(S), rule=lambda model, t,s: model.p_grid[t,s] + wind[t,s] + data['conversion_h2p']*model.e_h2p[t,s] - model.e_p2h[t,s] >= data['demand_schedule'][t_start+t])

    # contraints
    model.h_contraint = Constraint(range(T-1), range(S), rule=lambda model, t, s: model.h[t+1, s] == model.h[t, s] + data['conversion_p2h']*model.e_p2h[t, s] - model.e_h2p[t, s])

    model.p2h_constraint = Constraint(range(T), range(S), rule=lambda model, t, s:  model.e_h2p[t, s] <= model.h[t, s])
    model.p2h_constraint2 = Constraint(range(T),range(S),rule=lambda model,t,s: data['conversion_h2p']*model.e_h2p[t, s] <= data['h2p_max_rate'])
    
    model.conversion_contraint = Constraint(range(T), range(S), rule=lambda model, t, s:  data['conversion_p2h']*model.e_p2h[t, s] <= data['p2h_max_rate']*model.e_on[t-1, s])

    model.tank_start = Constraint(range(S), rule=lambda model, s: model.h[0, s] == h_0)

    model.electrolyser_start = Constraint(range(S), rule=lambda model, s: model.e_on[-1, s] == e_on_0)

    # Create a solver
    solver = SolverFactory('gurobi') 

    # Solve the model
    results = solver.solve(model, tee=False)
    return results,model

## Define the multi-stage stochastic programming policy

In [8]:
class MultiStagePolicy:
    def __init__(self, data, L=4,k=3):
        self.data = data
        self.L = L
        self.k = k

    def __call__(self, t, h, e_on, wind, wind_previous, price, price_previous,data):
        L = min(self.L, self.data['num_timeslots'] - t) - 1

        scenarios, scenario_probs = generate_scenarios(wind, price, wind_previous, price_previous, L, k=self.k)
        wind_matrix, price_matrix, a_set = scenarios_to_matrices(scenarios)
        results, model = solve_multi_stage_model(t, scenarios, a_set, scenario_probs[-1], price_matrix, wind_matrix, h, e_on)

        if not results.solver.termination_condition == TerminationCondition.optimal:
            raise ValueError(f"Optimal solution not found t={t}")
        
        e_on = value(model.e_on[0, 0]) if t < self.data['num_timeslots'] - 1 else 0
        e_p2h = value(model.e_p2h[0, 0])
        e_h2p = value(model.e_h2p[0, 0])
        p_grid = value(model.p_grid[0, 0])
        return e_on, e_p2h, e_h2p, p_grid

In [9]:
multi_stage_policy = MultiStagePolicy(data)
sim_MDP(10,multi_stage_policy,winds,prices)

Simulating MDP: 100%|██████████| 10/10 [00:18<00:00,  1.88s/it]


(447.05983344371873,
 [40.38463047549456,
  192.45521355444066,
  338.5858049880253,
  495.4855396093075,
  1003.7303000818753,
  231.48562018926754,
  599.3827889466544,
  853.8901713072801,
  558.2200093114479,
  156.97825597339354])